# Создание датасета для дообучения модели

Для начала нужно будет собрать датасет, содержащий в себе спортивные новости по разным видам спорта. Я буду использовать новости с сайта `championat.com`. На сайте есть новости по всем видам спорта.
Я использовал библиотеку `requests` и `BeatifulSoup4` для поиска нужных данных по `html` меткам страницы.

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

# Хеадер для запроса
my_headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:50.0) Gecko/20100101 Firefox/50.0'}

# Список видов спорта, которые мне нужны [название, кол-во страниц, номер первой страницы], страница ~ 65 новостей
sports = [['football', 1, 0], ['hockey', 1, 0], ['boxing', 1, 0], ['tennis', 1, 0], ['basketball', 1, 0], ['auto', 1, 0]]

# Файл для сохранения новостей
file_name =  'SportData.txt'

# Токены для разделения новостей
bos_token = '[SN]'
eos_token = '[EN]'
pad_token = '[PD]'

In [2]:
from ipywidgets import IntProgress
from IPython.display import display
from time import sleep
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Создание сессии для запросов
session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

Весь процесс сбора разделяется на 2 этапа. Сначала собираются ссылки на новости с новостной страницы, а затем из каждой новости достаётся текст статьи. Первая функция собирает все ссылки на статьи, а вторая пишет их текст в файл. Статьи начинаются с `[SN]` токена и заканчиваются `[EN]` токеном. Это нужно, чтобы нейросеть разделяла новости при генерации.

In [3]:
def urls_collector(name, pages_number, start_page):
  print(name, 'urls collector')
  progress_bar = IntProgress(min=0, max=pages_number, value=0)
  display(progress_bar)

  urls = []
  for i in range(pages_number):
    request = session.get('https://www.championat.com/news/' + name + '/' + str(i + 1 + start_page) + '.html', headers=my_headers)
  
    if request.status_code != 200:
      print('Request status code', request.status_code)
      continue
    
    page = bs(request.text, "html.parser")
    request_news = page.find_all('a', class_='news-item__title')
    for news in request_news:
      if news['href'].find('http') == -1:
        urls += [news['href']]

    progress_bar.value += 1

  print(len(urls), 'news collected')  

  return urls

In [6]:
from ipywidgets.widgets import Textarea
def arcticles_collector(name, urls):
  print(name, 'arcticles collector')
  progress_bar = IntProgress(min=0, max=len(urls), value=0) # instantiate the bar
  display(progress_bar)

  f = open(file_name, 'a', encoding='utf-8')

  for url in urls:
    request = session.get('https://www.championat.com' + url, headers=my_headers)
    if request.status_code != 200:
      print('Request status code', request.status_code)
      continue
    page = bs(request.text, "html.parser")
    article_text = bos_token + ' '


    title = page.find(class_='article-head__title')
    article_text += title.text.strip() + '.\n'

    article = page.find(class_='article-content')
    texts = article.findAll(name='p')
    for text in texts:
      if not(text.has_attr('class')):
        article_text += text.text.strip() + '\n'

    article_text += eos_token + '\n\n'

    f.write(article_text)
    progress_bar.value += 1
    
  print(len(urls), name, 'articles collected')

  f.close()

In [8]:
for sport in sports:
    pages_number = sport[1]
    name = sport[0]
    skip_pages = sport[2]

    urls = urls_collector(name, pages_number, skip_pages)
    arcticles_collector(name, urls)

    print('\n\n')

football urls collector


IntProgress(value=0, max=1)

65 news collected
football arcticles collector


IntProgress(value=0, max=65)

65 football articles collected



hockey urls collector


...

65 basketball articles collected



auto urls collector


IntProgress(value=0, max=1)

65 news collected
auto arcticles collector


IntProgress(value=0, max=65)

65 auto articles collected





# Итоговый датасет

Мы получили датасет из 390 статей в файле `SportData.txt`.

Для дообучнения `RuGPT3` использовался датасет из ~700 футольных статей, ~500 хоккейных, а также по 130 теннисных, боксёрских, гоночных и баскетбольных. Он находится в файле `TrainingDataset.txt`.

Количество статей указано примерное, так как полносью собранный датасет не помещался в память видеокарты и приходилось удалять часть статей. Также количетсво статей отличается в связи с различной популярностью разных видов спорта.